In [7]:

pip install pymdptoolbox


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pymdptoolbox: filename=pymdptoolbox-4.0b3-py3-none-any.whl size=25657 sha256=b664e675e2db3c33a47d2c8a1d7d5b41d55fefd34b338720b7ac9b99e2aa992e
  Stored in directory: /root/.cache/pip/wheels/09/a8/27/a76d688633fa5d71984c288499c2170a8d06726135b8e216fd
Successfully built pymdptoolbox


In [8]:
import mdptoolbox, mdptoolbox.example, mdptoolbox.util
import numpy as np


# The Cleaning Robot

Dead line : 11 nov for both groups (note book + report describing your mdp, including a drawing)

Consider a house-cleaning robot. It can be either in the living room or at its charging station - or out  of battery. 
The living room can be clean or dirty. So there are five states: LD(in the living room, dirty), LC(in the living room, clean), CD(at the charger, dirty), CC(at the charger, clean), O (out of power).


####  
When in the living room,    the robot  can either choose to clean or to charge. 

When in the charging station, the robot can either choose to clean or to keep charging.

When out of order, any of the two actions (clean, charge) leads to the same results: staying out of power

####  

The reward for being  in a clean state is rc 

The reward for being in a dirty state is rd < rc at each time step

The reward for being out of power is  costcrash  -  lower or equal to rd ; let us set it equal to rd  (the living rooms becomes dirty anyway)
 

####  

When  the robot decides to  clean,
*  if it is in the living room, then it may become out of power with proba e
*  if it is in the charging station, no risk of running out of power   
*  cleaning a clean floor leaves it clean
*  cleaning a dirty floor can sometimes fail (even is battery is ok) - let eps be the probability of fail of the cleaning
     
When  the robot decides to  charge,  action charge always takes the robot to the charging station. The  dirtiness of the room can go from clean to dirty with a probability  pDust  and it stays for sure at the dirty level if dirty



#### Goal

Model the problem by a (infinite horizon) MDP (describe your model in details) 

As joined the draw explaining the mdp, you could differents states and links them.


In [87]:
def matrice_transitions_reward(rc, rd, costcrash, e, eps, pDust):
    R =np.array(
        [[rd, rd],
        [rc, rc],
        [rd, rd],
        [rc, rc],
        [costcrash, costcrash]])
    

        #CLEAN     
    P = np.array(
         #LD  #LC  #CD  #CC  #O
        [[[eps,1-eps-e,0.,0.,e], #LD
        [0,1-e,0.,0.,e],         #LC
        [eps,1-eps,0.,0.,0.],    #CD
        [0.,1,0.,0.,0.],         #CC
        [0.,0.,0.,0.,1]],        #O
        
        #CHARGE
         #LD  #LC  #CD  #CC  #O
        [[ 0., 0., 1, 0, 0.],       #LD
         [ 0., 0., 0., 1, 0],       #LC
         [ 0., 0., 1, 0., 0.],       #CD
         [ 0., 0., pDust,1-pDust,0.],#CC
         [ 0., 0., 0., 0., 1]]      #O
        ])    
        
        
    return P, R


In [203]:
rd= -10
rc =10
costcrash = 10
e = 1
eps = 0
pDust = 0.2

P, R = matrice_transitions_reward(rc, rd, costcrash, e, eps, pDust)

finiteHorizon = mdptoolbox.mdp.PolicyIteration(P, R, 0.9)
#finiteHorizon = mdptoolbox.mdp.FiniteHorizon(P,R,0.9,1)
finiteHorizon.run()
finiteHorizon.setVerbose()
#finiteHorizon.policy[:,0]
finiteHorizon.policy
#LD LC CD CC O

(1, 1, 1, 1, 0)

Determine the best policy.

* When the probability of runing out of battery is high
> For e = 0.9 , we got this policy: (0, 0, 0, 0, 0)



* When it is low
>   For e = 0.1, we got (0, 0, 0, 0, 0) 




(explain the results)
<br> 

> We just have the same result even changing the probability of out  of power in all states and the robot chooses to clean.
Results can be explained by hte fact that in any of the 5 states, choosing to clean leads to the same reward : either in clean state or in O. In both, the reward is same and greater.
It could fail in the first state LD by it has a chance to get a great reward by cleaning than charging( charging leads to a dirty state ) 





</br>


* What policy is choosen if  the robot has a very good battery (i.e. the probability of being out  of charge is very low) ? 


>         When the   robot has a very good battery ( I set e = 0.1), we got this policy : (0, 0, 0, 0, 0)




* What policy is choosen if the  living room gets dirty   quickly  (i.e.  pdust increases) ? 
>  When pDust increases  ( pDust = 0.8 , 1 ), we got this policy : (0, 0, 0, 0, 0)







* When e=0.1 does the best policy depend of rc ? of pDust?  

    When e = 0.1, tests show that policy depends on rc at some level of rc rising. But not really of pDust


* What if costcrash  is independent of rc ?
> if costcrash is independent of rc, we could face 3 case:
 <br> * if rc > costcrash and this case in LC, the action is going to be chosen beacause it is well rewarded to go a clean state than in dirty one.
 <br> * rc < costcrash : In this case, cleaning is going to be chosen favouritely.

Paste your code

In [188]:
def bestPolicy(batterieProba):
  e = batterieProba
  finiteHorizon = mdptoolbox.mdp.PolicyIteration(P, R, 0.9)
  finiteHorizon.run()
  finiteHorizon.setVerbose()
  print(finiteHorizon.policy)



bestPolicy(e)

(0, 0, 0, 0, 0)



Optional: extend your program so as to take into consideration several levels of battery (high, medium, low, very low for instance)

In [191]:
batteries = ["high", "medium", "low", "very low"]

probaBatteries = [0.9, .05, 0.3, 0.1]

def baterieOption(batterielevel):
    if batterielevel == "high":
      e = probaBatteries[0]
    if batterielevel == "medium":
      e = probaBatteries[1]
    if batterielevel == "low":
      e = probaBatteries[2]
    if batterielevel == "very low":
      e = probaBatteries[3]
    bestPolicy(e)


baterieOption("high")

(0, 0, 0, 0, 0)
